In [ ]:
!pip install basedosdados==1.6.10b1

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import zipfile
from zipfile import ZipFile 
import basedosdados as bd

In [ ]:
uf_diretorio = bd.read_sql('SELECT nome, sigla FROM basedosdados.br_bd_diretorios_brasil.uf', billing_project_id='input-dados', reauth=True)
ufs = uf_diretorio['sigla'].to_list()

Downloading: 100%|██████████| 27/27 [00:00<00:00, 166.29rows/s]


In [ ]:
# Cria pasta no drive pra salvar particionado os arquivos
folders = ['etapa_ensino_serie', 'localizacao', 'sexo_raca_cor', 'tempo_ensino', 'faixa_etaria']
for f in folders:
  for ano in [*range(2007, 2023)]:
    for uf in ufs:
        directory = '/content/gdrive/MyDrive/basedosdados/br_inep_sinopse_educacao_basica/output/{}/ano={}/sigla_uf={}'.format(f, ano, uf)
        if not os.path.exists(directory):
          os.makedirs(directory)

In [ ]:
years = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

path_option = {}
for y in years:
    var = '/content/gdrive/MyDrive/basedosdados/br_inep_sinopse_educacao_basica/input'
    data = 'Sinopse_Estatistica_da_Educação_Basica_' + str(y) + '.xlsx'
    path = os.path.abspath(os.path.join('..', var))
    path_option[y] = os.path.join(path, data)

### etapa_ensino_serie

In [ ]:
sheets = ['Educação Infantil 1.5', 'Anos Iniciais 1.15', 'Anos Finais 1.20', 'Ensino Médio 1.25', 'Educação Profissional 1.30', 'EJA 1.34']

rename_ei = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Total': 'total', 'Federal': '1_federal',
            'Estadual': '1_estadual', 'Municipal': '1_municipal',
            'Privada': '1_privada',
            'Total.1': '2_total', 'Federal.1': '2_federal',
            'Estadual.1': '2_estadual',
            'Municipal.1': '2_municipal', 'Privada.1': '2_privada'}

rename_ai = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Total': '1_total', 'Federal': '1_federal',
            'Estadual': '1_estadual', 'Municipal': '1_municipal',
            'Privada': '1_privada',
            'Total.1': '2_total', 'Federal.1': '2_federal',
            'Estadual.1': '2_estadual',
            'Municipal.1': '2_municipal', 'Privada.1': '2_privada',
            'Total.2': '3_total', 'Federal.2': '3_federal',
            'Estadual.2': '3_estadual', 'Municipal.2': '3_municipal',
            'Privada.2': '3_privada',
            'Total.3': '4_total', 'Federal.3': '4_federal',
            'Estadual.3': '4_estadual',
            'Municipal.3': '4_municipal', 'Privada.3': '4_privada',
            'Total.4': '5_total', 'Federal.4': '5_federal',
            'Estadual.4': '5_estadual',
            'Municipal.4': '5_municipal', 'Privada.4': '5_privada'}

rename_af = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Total': '6_total', 'Federal': '6_federal',
            'Estadual': '6_estadual', 'Municipal': '6_municipal',
            'Privada': '6_privada',
            'Total.1': '7_total', 'Federal.1': '7_federal',
            'Estadual.1': '7_estadual',
            'Municipal.1': '7_municipal', 'Privada.1': '7_privada',
            'Total.2': '8_total', 'Federal.2': '8_federal',
            'Estadual.2': '8_estadual', 'Municipal.2': '8_municipal',
            'Privada.2': '8_privada',
            'Total.3': '9_total', 'Federal.3': '9_federal',
            'Estadual.3': '9_estadual',
            'Municipal.3': '9_municipal', 'Privada.3': '9_privada'}

rename_em = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Total': '10_total', 'Federal': '10_federal',
            'Estadual': '10_estadual', 'Municipal': '10_municipal',
            'Privada': '10_privada',
            'Total.1': '11_total', 'Federal.1': '11_federal',
            'Estadual.1': '11_estadual',
            'Municipal.1': '11_municipal', 'Privada.1': '11_privada',
            'Total.2': '12_total', 'Federal.2': '12_federal',
            'Estadual.2': '12_estadual', 'Municipal.2': '12_municipal',
            'Privada.2': '12_privada',
            'Total.3': '13_total', 'Federal.3': '13_federal',
            'Estadual.3': '13_estadual',
            'Municipal.3': '13_municipal', 'Privada.3': '13_privada',
            'Total.4': '14_total', 'Federal.4': '14_federal',
            'Estadual.4': '14_estadual',
            'Municipal.4': '14_municipal', 'Privada.4': '14_privada'}   
            
rename_ep = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Total': '7_total', 'Federal': '7_federal',
            'Estadual': '7_estadual', 'Municipal': '7_municipal',
            'Privada': '7_privada',
            'Total.1': '8_total', 'Federal.1': '8_federal',
            'Estadual.1': '8_estadual',
            'Municipal.1': '8_municipal', 'Privada.1': '8_privada',
            'Total.2': '9_total', 'Federal.2': '9_federal',
            'Estadual.2': '9_estadual', 'Municipal.2': '9_municipal',
            'Privada.2': '9_privada',
            'Total.3': '10_total', 'Federal.3': '10_federal',
            'Estadual.3': '10_estadual',
            'Municipal.3': '10_municipal', 'Privada.3': '10_privada',
            'Total.4': '11_total', 'Federal.4': '11_federal',
            'Estadual.4': '11_estadual',
            'Municipal.4': '11_municipal', 'Privada.4': '11_privada'}

rename_eja = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Total': '13_total', 'Federal': '13_federal',
            'Estadual': '13_estadual', 'Municipal': '13_municipal',
            'Privada': '13_privada',
            'Total.1': '14_total', 'Federal.1': '14_federal',
            'Estadual.1': '14_estadual',
            'Municipal.1': '14_municipal', 'Privada.1': '14_privada'}

ordem = ['ano', 'sigla_uf', 'id_municipio', 'rede', 'etapa_ensino', 'serie', 'quantidade_matricula']

id_vars = ['ano', 'sigla_uf', 'id_municipio']

list_df = []
for y in years:
  #if y == 2007:
    dict_df = {}
    for s in range(len(sheets)):
        dict_df[s] = pd.read_excel(path_option[y], sheet_name=sheets[s], skiprows=8, dtype=str)
        dict_df[s].drop(0, axis=0, inplace=True)
        dict_df[s]['Unnamed: 3'] = dict_df[s]['Unnamed: 3'].replace(' ', np.nan)
        dict_df[s].dropna(subset='Unnamed: 3', axis=0, inplace=True)
        dict_df[s]['Unnamed: 1'] = dict_df[s]['Unnamed: 1'].str.strip()
        dict_df[s] = pd.merge(dict_df[s], uf_diretorio, how='left', left_on=['Unnamed: 1'], right_on=['nome']).reset_index(drop=True)
        dict_df[s]['ano'] = y
        if s == 0:
          dict_df[s].rename(columns=rename_ei, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['1_federal', '1_estadual', '1_municipal', '1_privada', '2_federal', '2_estadual', '2_municipal', '2_privada'],
                               var_name='etapa_ensino_rede', value_name='quantidade_matricula')
          dict_df[s]['etapa_ensino'] = dict_df[s].etapa_ensino_rede.str.split('_').str[0]
          dict_df[s]['rede'] = dict_df[s].etapa_ensino_rede.str.split('_').str[-1]
          dict_df[s]['serie'] = ''
          dict_df[s] = dict_df[s][ordem]
        if s == 1:
          dict_df[s].rename(columns=rename_ai, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['1_federal', '1_estadual', '1_municipal', '1_privada', '2_federal', '2_estadual', '2_municipal', '2_privada',
                                           '3_federal', '3_estadual', '3_municipal', '3_privada', '4_federal', '4_estadual', '4_municipal',  '4_privada',
                                           '5_federal', '5_estadual', '5_municipal', '5_privada'],
                               var_name='serie_rede', value_name='quantidade_matricula')
          dict_df[s]['serie'] = dict_df[s].serie_rede.str.split('_').str[0]
          dict_df[s]['rede'] = dict_df[s].serie_rede.str.split('_').str[-1]
          dict_df[s]['etapa_ensino'] = '3'
          dict_df[s] = dict_df[s][ordem]
        if s == 2:
          dict_df[s].rename(columns=rename_af, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['6_federal', '6_estadual', '6_municipal', '6_privada', '7_federal', '7_estadual', '7_municipal', '7_privada',
                                           '8_federal', '8_estadual', '8_municipal', '8_privada', '9_federal', '9_estadual', '9_municipal',  '9_privada'],
                               var_name='serie_rede', value_name='quantidade_matricula')
          dict_df[s]['serie'] = dict_df[s].serie_rede.str.split('_').str[0]
          dict_df[s]['rede'] = dict_df[s].serie_rede.str.split('_').str[-1]
          dict_df[s]['etapa_ensino'] = '4'
          dict_df[s] = dict_df[s][ordem]
        if s == 3:
          dict_df[s].rename(columns=rename_em, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['10_federal', '10_estadual', '10_municipal', '10_privada', '11_federal', '11_estadual', '11_municipal', '11_privada',
                                           '12_federal', '12_estadual', '12_municipal', '12_privada', '13_federal', '13_estadual', '13_municipal',  '13_privada',
                                           '14_federal', '14_estadual', '14_municipal', '14_privada'],
                               var_name='serie_rede', value_name='quantidade_matricula')
          dict_df[s]['serie'] = dict_df[s].serie_rede.str.split('_').str[0]
          dict_df[s]['rede'] = dict_df[s].serie_rede.str.split('_').str[-1]
          dict_df[s]['etapa_ensino'] = '5'
          dict_df[s] = dict_df[s][ordem]
        if s == 4 and y != 2009:
          dict_df[s].rename(columns=rename_ep, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['7_federal', '7_estadual', '7_municipal', '7_privada', '8_federal', '8_estadual', '8_municipal', '8_privada',
                                           '9_federal', '9_estadual', '9_municipal', '9_privada', '10_federal', '10_estadual', '10_municipal', '10_privada',
                                           '11_federal', '11_estadual', '11_municipal', '11_privada'],
                               var_name='etapa_ensino_rede', value_name='quantidade_matricula')
          dict_df[s]['etapa_ensino'] = dict_df[s].etapa_ensino_rede.str.split('_').str[0]
          dict_df[s]['rede'] = dict_df[s].etapa_ensino_rede.str.split('_').str[-1]
          dict_df[s]['serie'] = ''
          dict_df[s] = dict_df[s][ordem]
        if s == 4 and y == 2009:
          dict_df[s].rename(columns=rename_ep, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['7_federal', '7_estadual', '7_municipal', '7_privada', '8_federal', '8_estadual', '8_municipal', '8_privada'],
                               var_name='etapa_ensino_rede', value_name='quantidade_matricula')
          dict_df[s]['etapa_ensino'] = dict_df[s].etapa_ensino_rede.str.split('_').str[0]
          dict_df[s]['rede'] = dict_df[s].etapa_ensino_rede.str.split('_').str[-1]
          dict_df[s]['serie'] = ''
          dict_df[s] = dict_df[s][ordem]
        if s == 5:
          dict_df[s].rename(columns=rename_eja, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['13_federal', '13_estadual', '13_municipal', '13_privada', '14_federal', '14_estadual', '14_municipal', '14_privada'],
                               var_name='etapa_ensino_rede', value_name='quantidade_matricula')
          dict_df[s]['etapa_ensino'] = dict_df[s].etapa_ensino_rede.str.split('_').str[0]
          dict_df[s]['rede'] = dict_df[s].etapa_ensino_rede.str.split('_').str[-1]
          dict_df[s]['serie'] = ''
          dict_df[s] = dict_df[s][ordem]            
    list_df.append(dict_df)

new_df = []
for i in range(len(years)):
    new_list = list(list_df[i].values())
    new_aux = pd.concat(new_list).reset_index(drop=True)
    new_df.append(new_aux)

for y in years:
  for i in range(len(years)):
    df_a= new_df[i]
    for uf in ufs:
          #df_a = df_a[ordem]
          df_auf = df_a[df_a['sigla_uf'] == uf]
          df_auf.drop(['ano','sigla_uf'], axis=1, inplace=True)
          exec("df_auf.to_csv('/content/gdrive/MyDrive/basedosdados/br_inep_sinopse_educacao_basica/output/etapa_ensino_serie/ano={}/sigla_uf={}/data.csv', index = False)".format(y, uf))

## localizacao

In [ ]:
sheets = ['Creche 1.6', 'Pré-Escola 1.10', '1.16', '1.21', '1.26', '1.31', '1.35', '1.40', '1.46']

dict_df_rename = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
                  'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
                  'Total': 'urbana_total', 'Federal': 'urbana_federal',
                  'Estadual': 'urbana_estadual', 'Municipal': 'urbana_municipal',
                  'Privada': 'urbana_privada', 'Total.1': 'rural_total', 'Federal.1': 'rural_federal',
                  'Estadual.1': 'rural_estadual', 'Municipal.1': 'rural_municipal', 'Privada.1': 'rural_privada'}

etapa_ensino = ['1', '2', '3', '4', '5', '6', '12', '15', '16']

ordem = ['ano', 'sigla_uf', 'id_municipio',  'rede', 'etapa_ensino', 'localizacao', 'quantidade_matricula']

value_vars = ['urbana_federal', 'urbana_estadual', 'urbana_municipal', 'urbana_privada', 
              'rural_federal', 'rural_estadual', 'rural_municipal', 'rural_privada']

list_df = []
for y in years:
  #if y == 2007:
    dict_df = {}
    for s in range(len(sheets)):
        dict_df[s] = pd.read_excel(path_option[y], sheet_name=sheets[s], skiprows=8, dtype='string')
        dict_df[s].drop(0, axis=0, inplace=True)
        dict_df[s]['Unnamed: 3'] = dict_df[s]['Unnamed: 3'].replace(' ', np.nan)
        dict_df[s].dropna(subset='Unnamed: 3', axis=0, inplace=True)
        dict_df[s]['Unnamed: 1'] = dict_df[s]['Unnamed: 1'].str.strip()
        dict_df[s] = pd.merge(dict_df[s], uf_diretorio, how='left', left_on=['Unnamed: 1'], right_on=['nome']).reset_index(drop=True)
        dict_df[s]['ano'] = y
        dict_df[s]['etapa_ensino'] = etapa_ensino[s]
        dict_df[s].rename(columns=dict_df_rename, inplace=True)
        dict_df[s] = pd.melt(dict_df[s],
                    id_vars=['sigla_uf', 'id_municipio', 'etapa_ensino', 'ano'],
                    value_vars=value_vars, var_name='localizacao_rede',
                    value_name='quantidade_matricula')
        dict_df[s]['localizacao'] = dict_df[s].localizacao_rede.str.split('_').str[0]
        dict_df[s]['rede'] = dict_df[s].localizacao_rede.str.split('_').str[-1]
        dict_df[s] = dict_df[s][ordem]
    list_df.append(dict_df)

new_df = []
for y in range(len(years)):
    new_list = list(list_df[y].values())
    new_aux = pd.concat(new_list).reset_index(drop=True)
    new_df.append(new_aux)

for i in range(len(years)):
  df_a= new_df[i]
  for uf in ufs:
    df_a = df_a[ordem]
    df_auf = df_a[df_a['sigla_uf'] == uf]
    df_auf.drop(['ano','sigla_uf'], axis=1, inplace=True)
    exec("df_auf.to_csv('/content/gdrive/MyDrive/basedosdados/br_inep_sinopse_educacao_basica/output/localizacao/ano={}/sigla_uf={}/data.csv', index = False)".format(y, uf))

## sexo_raca_cor

In [ ]:
sheets = ['1.7', '1.11', '1.17', '1.22', '1.27', '1.32', '1.36', '1.41', '1.47']

dict_df_rename = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
                  'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
                  'Não Declarada':'feminino_9', 'Branca':'feminino_1', 'Preta':'feminino_2',
                  'Parda':'feminino_3', 'Amarela':'feminino_4', 'Indígena':'feminino_5',
                  'Não Declarada.1':'masculino_9', 'Branca.1':'masculino_1', 'Preta.1':'masculino_2',
                  'Parda.1':'masculino_3', 'Amarela.1':'masculino_4', 'Indígena.1':'masculino_5'}

etapa_ensino = ['1', '2', '3', '4', '5', '6', '12', '15', '16']

ordem = ['ano', 'sigla_uf', 'id_municipio', 'etapa_ensino', 'sexo', 'raca_cor', 'quantidade_matricula']

value_vars = ['feminino_9', 'feminino_1', 'feminino_2', 'feminino_3', 'feminino_4', 'feminino_5',
              'masculino_9', 'masculino_1', 'masculino_2', 'masculino_3', 'masculino_4', 'masculino_5']

list_df = []
for y in years:
    dict_df = {}
    for s in range(len(sheets)):
        dict_df[s] = pd.read_excel(path_option[y], sheet_name=sheets[s], skiprows=8, dtype='string')
        dict_df[s].drop(0, axis=0, inplace=True)
        dict_df[s]['Unnamed: 3'] = dict_df[s]['Unnamed: 3'].replace(' ', np.nan)
        dict_df[s].dropna(subset='Unnamed: 3', axis=0, inplace=True)
        dict_df[s]['Unnamed: 1'] = dict_df[s]['Unnamed: 1'].str.strip()
        dict_df[s] = pd.merge(dict_df[s], uf_diretorio, how='left', left_on=['Unnamed: 1'], right_on=['nome']).reset_index(drop=True)
        dict_df[s]['ano'] = y
        dict_df[s]['etapa_ensino'] = etapa_ensino[s]
        dict_df[s].rename(columns=dict_df_rename, inplace=True)
        if y != 2009:
          dict_df[s] = pd.melt(dict_df[s],
                      id_vars=['sigla_uf', 'id_municipio', 'etapa_ensino', 'ano'],
                      value_vars=value_vars, var_name='sexo_raca_cor',
                      value_name='quantidade_matricula')
        if y == 2009:
           dict_df[s] = pd.melt(dict_df[s],
                      id_vars=['sigla_uf', 'id_municipio', 'etapa_ensino', 'ano'],
                      value_vars=['feminino_9', 'feminino_1', 'feminino_2', 'feminino_3', 'feminino_4', 'feminino_5', 'masculino_9', 'masculino_1'], 
                      var_name='sexo_raca_cor', value_name='quantidade_matricula')         
        dict_df[s]['sexo'] = dict_df[s].sexo_raca_cor.str.split('_').str[0]
        dict_df[s]['raca_cor'] = dict_df[s].sexo_raca_cor.str.split('_').str[-1]
        dict_df[s] = dict_df[s][ordem]
    list_df.append(dict_df)

new_df = []
for i in range(len(years)):
    new_list = list(list_df[i].values())
    new_aux = pd.concat(new_list).reset_index(drop=True)
    new_df.append(new_aux)

for i in range(len(years)):
  df_a= new_df[i]
  for uf in ufs:
    df_a = df_a[ordem]
    df_auf = df_a[df_a['sigla_uf'] == uf]
    df_auf.drop(['ano','sigla_uf'], axis=1, inplace=True)
    exec("df_auf.to_csv('/content/gdrive/MyDrive/basedosdados/br_inep_sinopse_educacao_basica/output/sexo_raca_cor/ano={}/sigla_uf={}/data.csv', index = False)".format(y, uf))

## tempo_ensino

In [ ]:
sheets = ['1.9', '1.13', '1.19', '1.24', '1.29', '1.44', '1.50']

dict_df_rename = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
                  'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
                  'Federal':'integral_federal', 'Estadual':'integral_estadual', 'Municipal':'integral_municipal',
                  'Privada':'integral_privada', 'Federal.1':'parcial_federal', 'Estadual.1':'parcial_estadual', 
                  'Municipal.1':'parcial_municipal', 'Privada.1':'parcial_privada'}

etapa_ensino = ['1', '2', '3', '4', '5', '15', '16']

ordem = ['ano', 'sigla_uf', 'id_municipio', 'rede', 'etapa_ensino', 'tempo_ensino', 'quantidade_matricula']

value_vars = ['integral_federal', 'integral_estadual', 'integral_municipal', 'integral_privada', 
              'parcial_federal', 'parcial_estadual', 'parcial_municipal', 'parcial_privada']

list_df = []
for y in years:
    dict_df = {}
    for s in range(len(sheets)):
        dict_df[s] = pd.read_excel(path_option[y], sheet_name=sheets[s], skiprows=8, dtype='string')
        dict_df[s].drop(0, axis=0, inplace=True)
        dict_df[s]['Unnamed: 3'] = dict_df[s]['Unnamed: 3'].replace(' ', np.nan)
        dict_df[s].dropna(subset='Unnamed: 3', axis=0, inplace=True)
        dict_df[s]['Unnamed: 1'] = dict_df[s]['Unnamed: 1'].str.strip()
        dict_df[s] = pd.merge(dict_df[s], uf_diretorio, how='left', left_on=['Unnamed: 1'], right_on=['nome']).reset_index(drop=True)
        dict_df[s]['ano'] = y
        dict_df[s]['etapa_ensino'] = etapa_ensino[s]
        dict_df[s].rename(columns=dict_df_rename, inplace=True)
        dict_df[s] = pd.melt(dict_df[s],
                    id_vars=['sigla_uf', 'id_municipio', 'etapa_ensino', 'ano'],
                    value_vars=value_vars, var_name='tempo',
                    value_name='quantidade_matricula')
        dict_df[s]['tempo_ensino'] = dict_df[s].tempo.str.split('_').str[0]
        dict_df[s]['rede'] = dict_df[s].tempo.str.split('_').str[-1]
        dict_df[s] = dict_df[s][ordem]
    list_df.append(dict_df)

new_df = []
for i in range(len(years)):
    new_list = list(list_df[i].values())
    new_aux = pd.concat(new_list).reset_index(drop=True)
    new_df.append(new_aux)

for i in range(len(years)):
    df_a= new_df[i]
    for uf in ufs:
        df_a = df_a[ordem]
        df_auf = df_a[df_a['sigla_uf'] == uf]
        df_auf.drop(['ano','sigla_uf'], axis=1, inplace=True)
        exec("df_auf.to_csv('/content/gdrive/MyDrive/basedosdados/br_inep_sinopse_educacao_basica/output/sexo_raca_cor/ano={}/sigla_uf={}/data.csv', index = False)".format(y, uf))

## faixa_etaria

In [ ]:
sheets = ['1.8', '1.12', '1.18', '1.23', '1.28', '1.33', '1.37']

rename_cr = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Unnamed: 4': 'total', 'Unnamed: 5': '1_1',
            'Unnamed: 6': '1_2', 'Unnamed: 7': '1_3'}

rename_pr = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Unnamed: 4': 'total', 'Unnamed: 5': '2_1',
            'Unnamed: 6': '2_2', 'Unnamed: 7': '2_3'}

rename_ai = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Unnamed: 4': 'total', 'Unnamed: 5': '3_4',
            'Unnamed: 6': '3_5', 'Unnamed: 7': '3_6',
            'Unnamed: 8': '3_7', 'Unnamed: 9': '3_8',
            'Unnamed: 10': '3_9'}

rename_af = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Unnamed: 4': 'total', 'Unnamed: 5': '4_10',
            'Unnamed: 6': '4_6', 'Unnamed: 7': '4_7',
            'Unnamed: 8': '4_8', 'Unnamed: 9': '4_11',
            'Unnamed: 10': '4_12'}

rename_em = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Unnamed: 4': 'total', 'Unnamed: 5': '5_13',
            'Unnamed: 6': '5_7', 'Unnamed: 7': '5_8',
            'Unnamed: 8': '5_11', 'Unnamed: 9': '5_12'}   
            
rename_ep = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Unnamed: 4': 'total', 'Unnamed: 5': '6_13',
            'Unnamed: 6': '6_7', 'Unnamed: 7': '6_8',
            'Unnamed: 8': '6_11', 'Unnamed: 9': '6_12'}

rename_eja = {'Unnamed: 0': 'regiao_geografica', 'sigla': 'sigla_uf',
            'Unnamed: 2': 'municipio', 'Unnamed: 3': 'id_municipio',
            'Unnamed: 4': 'total', 'Unnamed: 5': '7_13',
            'Unnamed: 6': '7_7', 'Unnamed: 7': '7_8',
            'Unnamed: 8': '7_11', 'Unnamed: 9': '7_14',
            'Unnamed: 10': '7_15', 'Unnamed: 11': '7_16', 'Unnamed: 12': '7_18'}

ordem = ['ano', 'sigla_uf', 'id_municipio', 'etapa_ensino', 'faixa_etaria', 'quantidade_matricula']

id_vars = ['ano', 'sigla_uf', 'id_municipio']

list_df = []
for y in years:
  #if y == 2007:
    dict_df = {}
    for s in range(len(sheets)):
        dict_df[s] = pd.read_excel(path_option[y], sheet_name=sheets[s], skiprows=8, dtype=str)
        dict_df[s].drop(0, axis=0, inplace=True)
        dict_df[s]['Unnamed: 3'] = dict_df[s]['Unnamed: 3'].replace(' ', np.nan)
        dict_df[s].dropna(subset='Unnamed: 3', axis=0, inplace=True)
        dict_df[s]['Unnamed: 1'] = dict_df[s]['Unnamed: 1'].str.strip()
        dict_df[s] = pd.merge(dict_df[s], uf_diretorio, how='left', left_on=['Unnamed: 1'], right_on=['nome']).reset_index(drop=True)
        dict_df[s]['ano'] = y
        if s == 0:
          dict_df[s].rename(columns=rename_cr, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['1_1', '1_2', '1_3'],
                               var_name='etapa_ensino_rede', value_name='quantidade_matricula')
          dict_df[s]['etapa_ensino'] = dict_df[s].etapa_ensino_rede.str.split('_').str[0]
          dict_df[s]['faixa_etaria'] = dict_df[s].etapa_ensino_rede.str.split('_').str[-1]
          dict_df[s] = dict_df[s][ordem]
        if s == 1:
          dict_df[s].rename(columns=rename_pr, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['2_1', '2_2', '2_3'],
                               var_name='serie_rede', value_name='quantidade_matricula')
          dict_df[s]['etapa_ensino'] = dict_df[s].serie_rede.str.split('_').str[0]
          dict_df[s]['faixa_etaria'] = dict_df[s].serie_rede.str.split('_').str[-1]
          dict_df[s] = dict_df[s][ordem]
        if s == 2:
          dict_df[s].rename(columns=rename_ai, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['3_4', '3_5', '3_6', '3_7', '3_8', '3_9'],
                               var_name='serie_rede', value_name='quantidade_matricula')
          dict_df[s]['etapa_ensino'] = dict_df[s].serie_rede.str.split('_').str[0]
          dict_df[s]['faixa_etaria'] = dict_df[s].serie_rede.str.split('_').str[-1]
          dict_df[s] = dict_df[s][ordem]
        if s == 3:
          dict_df[s].rename(columns=rename_af, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['4_10', '4_6', '4_7', '4_8', '4_11', '4_12'],
                               var_name='serie_rede', value_name='quantidade_matricula')
          dict_df[s]['etapa_ensino'] = dict_df[s].serie_rede.str.split('_').str[0]
          dict_df[s]['faixa_etaria'] = dict_df[s].serie_rede.str.split('_').str[-1]
          dict_df[s] = dict_df[s][ordem]
        if s == 4:
          dict_df[s].rename(columns=rename_em, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['5_13', '5_7', '5_8', '5_11', '5_12'],
                               var_name='etapa_ensino_rede', value_name='quantidade_matricula')
          dict_df[s]['etapa_ensino'] = dict_df[s].etapa_ensino_rede.str.split('_').str[0]
          dict_df[s]['faixa_etaria'] = dict_df[s].etapa_ensino_rede.str.split('_').str[-1]
          dict_df[s] = dict_df[s][ordem]
        if s == 5:
          dict_df[s].rename(columns=rename_ep, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['6_13', '6_7', '6_8', '6_11', '6_12'],
                               var_name='etapa_ensino_rede', value_name='quantidade_matricula')
          dict_df[s]['etapa_ensino'] = dict_df[s].etapa_ensino_rede.str.split('_').str[0]
          dict_df[s]['faixa_etaria'] = dict_df[s].etapa_ensino_rede.str.split('_').str[-1]
          dict_df[s] = dict_df[s][ordem]            
        if s == 6:
          dict_df[s].rename(columns=rename_eja, inplace=True)
          dict_df[s] = pd.melt(dict_df[s], id_vars=id_vars, 
                               value_vars=['7_13', '7_7', '7_8', '7_11', '7_14', '7_15', '7_16', '7_18'],
                               var_name='etapa_ensino_rede', value_name='quantidade_matricula')
          dict_df[s]['etapa_ensino'] = dict_df[s].etapa_ensino_rede.str.split('_').str[0]
          dict_df[s]['faixa_etaria'] = dict_df[s].etapa_ensino_rede.str.split('_').str[-1]
          dict_df[s] = dict_df[s][ordem]            
    list_df.append(dict_df)

new_df = []
for i in range(len(years)):
    new_list = list(list_df[i].values())
    new_aux = pd.concat(new_list).reset_index(drop=True)
    new_df.append(new_aux)

# for y in years:
#   for i in range(len(years)):
#     df_a= new_df[i]
#     for uf in ufs:
#           #df_a = df_a[ordem]
#           df_auf = df_a[df_a['sigla_uf'] == uf]
#           df_auf.drop(['ano','sigla_uf'], axis=1, inplace=True)
#           exec("df_auf.to_csv('/content/gdrive/MyDrive/basedosdados/br_inep_sinopse_educacao_basica/output/etapa_ensino_serie/ano={}/sigla_uf={}/data.csv', index = False)".format(y, uf))